## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from confeeg import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Rawson,-43.3002,-65.1023,61.72,78,100,22.64,AR,overcast clouds
1,1,Carnarvon,-24.8667,113.6333,87.80,62,0,10.36,AU,clear sky
2,2,Provideniya,64.3833,-173.3000,25.07,93,100,11.25,RU,light snow
3,3,Erenhot,43.6475,111.9767,25.59,19,0,15.35,CN,clear sky
4,4,Avarua,-21.2078,-159.7750,82.40,78,20,6.91,CK,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Carnarvon,AU,87.80,clear sky,-24.8667,113.6333,
4,Avarua,CK,82.40,few clouds,-21.2078,-159.7750,
7,San Eugenio,PH,83.41,overcast clouds,16.3598,120.3570,
8,Basco,PH,76.21,broken clouds,20.4487,121.9702,
11,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,
12,Sassandra,CI,77.49,overcast clouds,4.9500,-6.0833,
13,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
14,Georgetown,MY,89.60,few clouds,5.4112,100.3354,
18,Labuhan,ID,83.82,overcast clouds,-6.8844,112.2051,
21,Hithadhoo,MV,84.38,broken clouds,-0.6000,73.0833,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Carnarvon,AU,87.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
4,Avarua,CK,82.40,few clouds,-21.2078,-159.7750,Paradise Inn
7,San Eugenio,PH,83.41,overcast clouds,16.3598,120.3570,Albans Lodging House
8,Basco,PH,76.21,broken clouds,20.4487,121.9702,Nathaniel's Lodge
11,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Sassandra,CI,77.49,overcast clouds,4.9500,-6.0833,la terrasse
13,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
14,Georgetown,MY,89.60,few clouds,5.4112,100.3354,Cititel Penang
18,Labuhan,ID,83.82,overcast clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C
21,Hithadhoo,MV,84.38,broken clouds,-0.6000,73.0833,Scoop Guest House


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=290, point_radius=4)


In [48]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))